# Сегментация предложений
## *Ф.М. Достоевский "Братья Карамазовы"*

In [1]:
import os
from nltk import sent_tokenize
import re
import pandas as pd
from pprint import pprint
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer

### 1. Выгрузка данных и предварительная обработка

In [2]:
with open('brothers_karamazov.txt', 'r', encoding='utf-8') as f:
      full_text = f.read()

In [3]:
splitted_text = [re.sub('[\n\t]', ' ', x) for x in sent_tokenize(full_text)[100:150]]
splitted_text #отрывок, поделенный на предложения

['Об этом я теперь распространяться не стану, тем более что много еще придется рассказывать об этом первенце Федора Павловича, а теперь лишь ограничиваюсь самыми необходимыми о нем сведениями, без которых мне и романа начать невозможно.',
 'Во-первых, этот Дмитрий Федорович был один только из трех сыновей Федора Павловича, который рос в убеждении, что он все же имеет некоторое состояние и когда достигнет совершенных лет, то будет независим.',
 'Юность и молодость его протекли беспорядочно: в гимназии он не доучился, попал потом в одну военную школу, потом очутился на Кавказе, выслужился, дрался на дуэли, был разжалован, опять выслужился, много кутил и сравнительно прожил довольно денег.',
 'Стал же получать их от Федора Павловича не раньше совершеннолетия, а до тех пор наделал долгов.',
 'Федора Павловича, отца своего, узнал и увидал в первый раз уже после совершеннолетия, когда нарочно прибыл в наши места объясниться с ним насчет своего имущества.',
 'Кажется, родитель ему и тогда не 

**Как можно заметить, в тексте присутствуют лишние знаки. Уберем их с помощью регулярного выражения:**

In [4]:
splitted_text = [re.sub(r'III  ', '', sent) for sent in splitted_text]
splitted_text = [re.sub(r'\xa0', '', sent) for sent in splitted_text]
splitted_text

['Об этом я теперь распространяться не стану, тем более что много еще придется рассказывать об этом первенце Федора Павловича, а теперь лишь ограничиваюсь самыми необходимыми о нем сведениями, без которых мне и романа начать невозможно.',
 'Во-первых, этот Дмитрий Федорович был один только из трех сыновей Федора Павловича, который рос в убеждении, что он все же имеет некоторое состояние и когда достигнет совершенных лет, то будет независим.',
 'Юность и молодость его протекли беспорядочно: в гимназии он не доучился, попал потом в одну военную школу, потом очутился на Кавказе, выслужился, дрался на дуэли, был разжалован, опять выслужился, много кутил и сравнительно прожил довольно денег.',
 'Стал же получать их от Федора Павловича не раньше совершеннолетия, а до тех пор наделал долгов.',
 'Федора Павловича, отца своего, узнал и увидал в первый раз уже после совершеннолетия, когда нарочно прибыл в наши места объясниться с ним насчет своего имущества.',
 'Кажется, родитель ему и тогда не 

In [5]:
text = ' '.join(splitted_text)
text #исходный отрывок, не поделенный на предложения

'Об этом я теперь распространяться не стану, тем более что много еще придется рассказывать об этом первенце Федора Павловича, а теперь лишь ограничиваюсь самыми необходимыми о нем сведениями, без которых мне и романа начать невозможно. Во-первых, этот Дмитрий Федорович был один только из трех сыновей Федора Павловича, который рос в убеждении, что он все же имеет некоторое состояние и когда достигнет совершенных лет, то будет независим. Юность и молодость его протекли беспорядочно: в гимназии он не доучился, попал потом в одну военную школу, потом очутился на Кавказе, выслужился, дрался на дуэли, был разжалован, опять выслужился, много кутил и сравнительно прожил довольно денег. Стал же получать их от Федора Павловича не раньше совершеннолетия, а до тех пор наделал долгов. Федора Павловича, отца своего, узнал и увидал в первый раз уже после совершеннолетия, когда нарочно прибыл в наши места объясниться с ним насчет своего имущества. Кажется, родитель ему и тогда не понравился; пробыл он

### 2. Метрики

In [6]:
def metrics (gold, method, regexp=None):
    tp = 0
    fp = 0
    fn = 0
    
    if method == 'regexp':
        len_func = lambda sent, regexp: len(re.split(regexp, sent))
    elif method == 'punkt':
        len_func = lambda sent, regexp: len(tokenizer.tokenize(sent))
    else:
        raise ValueError('incorrect method')
    
    for sent in gold:
        if len_func(sent, regexp) == 1:
            tp += 1
        else:
            fp += 1

    for i in range(len(gold)-1):
        sent1, sent2 = gold[i], gold[i+1]
        sent = ' '.join([sent1, sent2])
        if len_func(sent, regexp) == 2:
            tp += 1
        else:
            fn += 1

    precision = (tp/(tp+fp))
    recall = (tp/(tp+fn))
    f1 = 2*(precision*recall)/(precision+recall)
    print('Precision - ', precision)
    print('Recall - ', recall)
    print('F1 - ', f1)

### 3. Сегментация текста с помощью регулярного выражения

In [7]:
regexp1 = '(?<=[\.\?\!])\s(?=[А-ЯЁ])'
parsed_text = re.split(regexp1, text)
parsed_text[:5]

['Об этом я теперь распространяться не стану, тем более что много еще придется рассказывать об этом первенце Федора Павловича, а теперь лишь ограничиваюсь самыми необходимыми о нем сведениями, без которых мне и романа начать невозможно.',
 'Во-первых, этот Дмитрий Федорович был один только из трех сыновей Федора Павловича, который рос в убеждении, что он все же имеет некоторое состояние и когда достигнет совершенных лет, то будет независим.',
 'Юность и молодость его протекли беспорядочно: в гимназии он не доучился, попал потом в одну военную школу, потом очутился на Кавказе, выслужился, дрался на дуэли, был разжалован, опять выслужился, много кутил и сравнительно прожил довольно денег.',
 'Стал же получать их от Федора Павловича не раньше совершеннолетия, а до тех пор наделал долгов.',
 'Федора Павловича, отца своего, узнал и увидал в первый раз уже после совершеннолетия, когда нарочно прибыл в наши места объясниться с ним насчет своего имущества.']

In [8]:
len (parsed_text)

48

**На два предложения меньше, чем надо. Посмотрим, где ошибается регулярное выражение:**

In [9]:
for i in range(len(splitted_text)):
    if i == 48:
        break
    if not splitted_text[i] in parsed_text:
        print(f'Верное: {splitted_text[i]}')
        print(f'Неверное: {parsed_text[i]}')
        print('\n')

Верное: Федор Павлович не взял в этот раз ни гроша, потому что генеральша рассердилась, ничего не дала и, сверх того, прокляла их обоих; но он и не рассчитывал на этот раз взять, а прельстился лишь замечательною красотой невинной девочки и, главное, ее невинным видом, поразившим его, сладострастника и доселе порочного любителя лишь грубой женской красоты.
Неверное: Федор Павлович не взял в этот раз ни гроша, потому что генеральша рассердилась, ничего не дала и, сверх того, прокляла их обоих; но он и не рассчитывал на этот раз взять, а прельстился лишь замечательною красотой невинной девочки и, главное, ее невинным видом, поразившим его, сладострастника и доселе порочного любителя лишь грубой женской красоты. «Меня эти невинные глазки как бритвой тогда по душе полоснули»,– говаривал он потом, гадко по-своему хихикая.


Верное: «Меня эти невинные глазки как бритвой тогда по душе полоснули»,– говаривал он потом, гадко по-своему хихикая.
Неверное: Впрочем, у развратного человека и это могл

**Регулярное выражение не сегментирует предложение, являющееся прямой речью, т.к. в данном случае оно начинается не на заглавную букву, а на знак кавычки. Попробуем составить другое регулярное выражение, допускающее и этот случай:**

In [10]:
regexp2 = '(?<=[\.\?\!])\s(?=[^а-яё])'
parsed_text = re.split(regexp2, text)
parsed_text[:5]

['Об этом я теперь распространяться не стану, тем более что много еще придется рассказывать об этом первенце Федора Павловича, а теперь лишь ограничиваюсь самыми необходимыми о нем сведениями, без которых мне и романа начать невозможно.',
 'Во-первых, этот Дмитрий Федорович был один только из трех сыновей Федора Павловича, который рос в убеждении, что он все же имеет некоторое состояние и когда достигнет совершенных лет, то будет независим.',
 'Юность и молодость его протекли беспорядочно: в гимназии он не доучился, попал потом в одну военную школу, потом очутился на Кавказе, выслужился, дрался на дуэли, был разжалован, опять выслужился, много кутил и сравнительно прожил довольно денег.',
 'Стал же получать их от Федора Павловича не раньше совершеннолетия, а до тех пор наделал долгов.',
 'Федора Павловича, отца своего, узнал и увидал в первый раз уже после совершеннолетия, когда нарочно прибыл в наши места объясниться с ним насчет своего имущества.']

In [11]:
len (parsed_text)

50

**Количество предложений совпадает! Сравним с помощью метрики:**

In [12]:
metrics(parsed_text, 'regexp', regexp=regexp1)
print('\n')
metrics(parsed_text, 'regexp', regexp=regexp2)

Precision -  1.0
Recall -  0.9797979797979798
F1 -  0.989795918367347


Precision -  1.0
Recall -  1.0
F1 -  1.0


### 4. Обучение PunktSentenceTokenizer

In [13]:
trainer = PunktTrainer()
trainer.INCLUDE_ALL_COLLOCS = True
trainer.train(full_text)
 
tokenizer = PunktSentenceTokenizer(trainer.get_params())
print(tokenizer._params.abbrev_types)

{'т', 'решу', 'фр', 'ею»', 'ст', 'nb', 'лад', 'рае', 'гл', 'p', 'к.»', 'проч', 'ss', 'qua', 'тя»', 'он»', 's', 'dei', 'лат', 'д', 'вою', 'р', 'шаль', 'хер', 'то»', 'рр'}


### 5. Применение PunktSentenceTokenizer к выбранному отрывку

In [14]:
tokenized_text = tokenizer.tokenize(text)
tokenized_text[:5]

['Об этом я теперь распространяться не стану, тем более что много еще придется рассказывать об этом первенце Федора Павловича, а теперь лишь ограничиваюсь самыми необходимыми о нем сведениями, без которых мне и романа начать невозможно.',
 'Во-первых, этот Дмитрий Федорович был один только из трех сыновей Федора Павловича, который рос в убеждении, что он все же имеет некоторое состояние и когда достигнет совершенных лет, то будет независим.',
 'Юность и молодость его протекли беспорядочно: в гимназии он не доучился, попал потом в одну военную школу, потом очутился на Кавказе, выслужился, дрался на дуэли, был разжалован, опять выслужился, много кутил и сравнительно прожил довольно денег.',
 'Стал же получать их от Федора Павловича не раньше совершеннолетия, а до тех пор наделал долгов.',
 'Федора Павловича, отца своего, узнал и увидал в первый раз уже после совершеннолетия, когда нарочно прибыл в наши места объясниться с ним насчет своего имущества.']

In [15]:
len (tokenized_text)

50

**Количество предложений совпадает! Сравним с помощью метрики:**

In [16]:
metrics(tokenized_text, 'punkt')

Precision -  1.0
Recall -  1.0
F1 -  1.0
